<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/MasterPlanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Planning

## Implementierung

In [5]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 28.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.18.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
google-colab 1.0.0 requ

In [6]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

In [7]:
! git clone https://github.com/AlexKressner/Industrielles_Management

fatal: destination path 'Industrielles_Management' already exists and is not an empty directory.


In [8]:
path = "Industrielles_Management/Daten/MasterPlanning"

In [9]:
# Kapazitäten
beschaffung_kapa = pd.read_csv(f"{path}/Beschaffungskapazitäten.csv", sep=";")
produktion_kapa = pd.read_csv(f"{path}/Produktionskapazitäten.csv", sep=";")

In [10]:
# Kosten
produktion_kosten = pd.read_csv(f"{path}/Produktionskosten.csv", sep=";", decimal=",")
beschaffung_kosten = pd.read_csv(f"{path}/Beschaffungskosten.csv", sep=";", decimal=",")
lager_kosten = pd.read_csv(f"{path}/Lagerkosten.csv", sep=";", decimal=",")
transport_beschaffung_kosten = pd.read_csv(f"{path}/Transportkosten_Beschaffung.csv", sep=";", decimal=",")
transport_distribution_kosten = pd.read_csv(f"{path}/Transportkosten_Distribution.csv", sep=";", decimal=",")

In [11]:
# Direktbedarfskoeffizienten
koeffizienten = pd.read_csv(f"{path}/Direktbedarfskoeffizienten.csv", sep=";", decimal=",")

In [12]:
# Nachfrage
nachfrage = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

## Indexmengen

In [13]:
I = produktion_kapa["Werk"].unique().tolist() # Menge der Produktionsstandorte

In [14]:
J = nachfrage["Markt"].unique().tolist() #Menge der Märkte

In [15]:
L = beschaffung_kapa["Lieferant"].unique().tolist() # Menge der Lieferanten

In [16]:
P = lager_kosten["Produkt"].unique().tolist() # Menge der Produkte

In [17]:
VP = koeffizienten["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

In [18]:
FP = koeffizienten["Fertigprodukt"].unique().tolist() # Menge der Fertigprodukte

In [19]:
A = {}
for vp in koeffizienten["Vorprodukt"].unique():
  A[vp] = koeffizienten[koeffizienten["Vorprodukt"]==vp]["Fertigprodukt"].tolist()

In [20]:
T = beschaffung_kapa["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Beschaffungsmengen der jeweiligen Vorprodukte (Flasche und Vitaminwasser)
Z={}
for l in L:
  for i in I:
    for p in VP:
      for t in T:
        Z[l,i,p,t] = solver.NumVar(0.0, infinity, f"{l},{i},{p},{t}")

In [ ]:
# Produktionsmenge Fertigprodukte
X={}
for i in I:
  for t in T:
    for p in FP:
      X[i,p,t] = solver.NumVar(0.0, infinity, f"{i},{p},{t}")

In [ ]:
# Distributionsmenge Fertigprodukte
Y={}
for i in I:
  for j in J:
    for t in T:
      for p in FP:
        Y[i,j,p,t] = solver.NumVar(0.0, infinity, f"{i},{j},{p},{t}")

In [ ]:
# Bestand von Produkt (Vor- und Fertigprodukt)
B={}
for i in I:
  for p in P:
    for t in T:
      B[i,p,t] = solver.NumVar(0.0, infinity, f"{i},{p},{t}")

In [ ]:
# Nutzung von Zusatzkapazitäten
O={}
for i in I:
    for t in T:
      O[i,t] = solver.NumVar(0.0, infinity, f"{i},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 360


## Parameter

In [ ]:
# Produktionskosten
pc = produktion_kosten.set_index(["Werk","Produkt"]).to_dict("dict")["Produktionskosten"] # Kosten reguläre Produktion in €/ME
oc = produktion_kosten.set_index(["Werk"]).to_dict("dict")["Kosten_pro_Zusatzkapa"] # Kosten Zusatzkapazitäten in €/Kapazitätseinheit

In [ ]:
# Beschaffungskosten
beschaffung_kosten.set_index(["Lieferant","Produkt"], inplace=True)
bc = beschaffung_kosten.to_dict("dict")["Beschaffungskosten"] # Kosten der Beschaffung eines Produktes in €/ME

In [ ]:
# Transportkosten
transport_beschaffung_kosten.set_index(["Lieferant","Werk","Produkt"], inplace=True)
btc = transport_beschaffung_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Beschaffung in €/ME
transport_distribution_kosten.set_index(["Werk","Markt","Produkt"], inplace=True)
dtc = transport_distribution_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Distribution in €/ME

In [ ]:
# Lagerkosten
lager_kosten.set_index(["Produkt"], inplace=True)
lc = lager_kosten.to_dict("dict")["Lagerkosten"] # Kosten Lagerung in €/Woche/ME

In [ ]:
# Nachfrage
nachfrage.set_index(["Markt","Produkt","Woche"], inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [ ]:
# Direktbedarfskoeffiziente
koeffizienten.set_index(["Vorprodukt","Fertigprodukt"], inplace=True)
a = koeffizienten.to_dict("dict")["Direktbedarfskoeffizient"]

In [ ]:
# Kapazitäten
produktion_kapa.set_index(["Werk","Woche"], inplace=True)
pcap = produktion_kapa.to_dict("dict")["Kapaztiät"]
pcap_plus = produktion_kapa.to_dict("dict")["Max_Kapa_durch_Zusatzschichten"]
beschaffung_kapa.set_index(["Lieferant","Produkt","Woche"], inplace=True)
bcap = beschaffung_kapa.to_dict("dict")["Kapazität"]

## Zielfunktion

### Zielfunktion
Min $K = \sum_{l,i,p \in VP,t} bc_{lp} * Z_{lipt} + \sum_{l,i,p \in VP,t} tbc_{lip} * Z_{lipt} + \sum_{i,p \in FP,t} pc_{ip} * X_{ipt} + \sum_{i,p,t} lc_{p} * B_{ipt} + \sum_{i,t} oc^+_{it} * O_{it} + \sum_{i,j,p,t} dtc_{ijp} * Y_{ijpt}$

In [ ]:
bc.get(('ErpoPlast', 'Vitaminwasser'),0)

0

In [ ]:
# Minimierung der gesamten Kosten
solver.Minimize(
    sum(bc.get((l,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Kosten Beschaffung
    sum(btc.get((l,i,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Transportkosten Beschaffung
    sum(pc[i,p]*X[i,p,t] for i in I for p in FP for t in T) + # Kosten der Produktion mit regulärer Kapazität
    sum(oc[i]*O[i,t] for i in I for t in T) + # Kosten der Produktion mit zusätzlicher Kapazität
    sum(lc[p]*B[i,p,t] for i in I for p in P for t in T) + # Kosten der Lagerung
    sum(dtc[i,j,p]*Y[i,j,p,t] for i in I for j in J for p in FP for t in T) # Transportkosten Distribution
    )

## Nebenbedingungen

**(1) Lieferantenkapazitäten**

$\sum_{i} Z_{lipt} \le bcap_{lpt}$

$∀ l,p \in VP, t$

In [ ]:
for l in L:
  for p in VP:
    for t in T:
      solver.Add(sum(Z[l,i,p,t] for i in I)<= bcap.get((l,p,t),0))

**(2) Produktionskapazitäten**

$\sum_{p \in FP} X_{ipt} \le pcap_{it} + O_{it}$

$∀ i, t$

In [ ]:
for i in I:
  for t in T:
    solver.Add(sum(X[i,p,t] for p in FP) <= pcap[i,t] + O[i,t])

**(3) Beschränkung der Zusatzkapazitäten**

$O_{it} \le pcap^+_{it}$

$∀ i, t$

In [ ]:
for i in I:
  for t in T:
    solver.Add(O[i,t] <= pcap_plus[i,t])

**(4) Lagerbilanzgleichung für die Fertigprodukte**

$B_{ipt} = B_{ip,t-1} + X_{ipt} - \sum_j Y_{ijpt}$

$∀ i,p \in FP, t$

In [ ]:
for i in I:
  for p in FP:
    for t in T:
        solver.Add(B[i,p,t] == B.get((i,p,t-1),0) + X[i,p,t] - sum(Y[i,j,p,t] for j in J))

**(5) Lagerbilanzgleichung für die Vorprodukte**

$B_{ipt} = B_{ip,t-1} + \sum_l Z_{lipt} - \sum_{p' \in A_{p} } a_{pp'} * X_{ip't}$

$∀ i,p \in VP, t$

In [ ]:
A["Flasche"]

['Well']

In [ ]:
for i in I:
  for p in VP:
    for t in T:
      solver.Add(B[i,p,t] == B.get((i,p,t-1),0) + sum(Z[l,i,p,t] for l in L) - sum(a[p,pp] * X[i,pp,t] for pp in A[p]))

**(6) Nachfragebefriedigung**

$\sum_i Y_{ijpt} = d_{jpt}$

$∀ j,p \in FP, t$

In [ ]:
for j in J:
  for t in T:
    for p in FP:
      solver.Add(sum(Y[i,j,p,t] for i in I) == d[j,p,t])

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 30257.260000000002


In [ ]:
for t in T:
  print(f"Woche: {t}")
  for i in I:
    for p in FP:
      val = round(X[i,p,t].solution_value())
    if val > 0:
      print(f"Werk {i}")
      print(f"Produktion: {val}")
    print(f"Fertigwarenbestand: {round(B[i,p,t].solution_value())}")
    print("\t")
  print("\n")


Woche: 1
Werk Curitiba
Produktion: 82
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 250
Fertigwarenbestand: 42
	
Werk East London
Produktion: 250
Fertigwarenbestand: 0
	


Woche: 2
Werk Curitiba
Produktion: 72
Fertigwarenbestand: 2
	
Werk Quanzhou
Produktion: 253
Fertigwarenbestand: 10
	
Werk East London
Produktion: 235
Fertigwarenbestand: 0
	


Woche: 3
Werk Curitiba
Produktion: 78
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 168
Fertigwarenbestand: 0
	
Werk East London
Produktion: 223
Fertigwarenbestand: 0
	


Woche: 4
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 212
Fertigwarenbestand: 22
	
Werk East London
Produktion: 218
Fertigwarenbestand: 0
	


Woche: 5
Werk Curitiba
Produktion: 123
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 150
Fertigwarenbestand: 0
	
Werk East London
Produktion: 275
Fertigwarenbestand: 0
	


